In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

In [2]:
caltech_dir = "C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train/"
categories = ["a0","a1","b0","b1","c0","c1"]
nb_classes = len(categories)

image_w = 60
image_h = 60

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("LA")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식

X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/numpy_data/multi_image_data.npy", xy)

print("ok", len(y))

a0  파일 길이 :  96
a0  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//a0\1.jpg
a1  파일 길이 :  37
a1  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//a1\136.jpg
b0  파일 길이 :  43
b0  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//b0\147.jpg
b1  파일 길이 :  37
b1  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//b1\163.jpg
c0  파일 길이 :  36
c0  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//c0\167.jpg
c1  파일 길이 :  16
c1  :  C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/img_others/train//c1\178.jpg
ok 265


In [3]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

X_train, X_test, y_train, y_test = np.load('C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/numpy_data/multi_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

Using TensorFlow backend.
C:\Users\dahyun\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dahyun\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dahyun\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dahyun\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

(198, 60, 60, 2)
198


In [4]:
categories = ["a0","a1","b0","b1","c0","c1"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [60]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(16, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = 'C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 60, 60, 16)        304       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 30, 30, 16)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 30, 30, 64)        9280      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 15, 128)       73856     
__________

In [62]:
#데이터셋이 적어서 validation을 그냥 test 데이터로 했습니다. 
#데이터셋이 충분하시면 이렇게 하시지 마시고 validation_split=0.2 이렇게 하셔서 테스트 셋으로 나누시길 권장합니다. 
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test) , callbacks=[checkpoint,
                                                                                                              early_stopping])

Train on 198 samples, validate on 67 samples
Epoch 1/50
198/198 [==============================] - 10s 51ms/step - loss: 1.9404 - acc: 0.2778 - val_loss: 1.7625 - val_acc: 0.3134

Epoch 00001: val_loss improved from inf to 1.76252, saving model to C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/model/multi_img_classification.model
Epoch 2/50
198/198 [==============================] - 3s 13ms/step - loss: 1.6995 - acc: 0.3636 - val_loss: 1.7713 - val_acc: 0.3134

Epoch 00002: val_loss did not improve from 1.76252
Epoch 3/50
198/198 [==============================] - 3s 13ms/step - loss: 1.6681 - acc: 0.3737 - val_loss: 1.7370 - val_acc: 0.3134

Epoch 00003: val_loss improved from 1.76252 to 1.73702, saving model to C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/model/multi_img_classification.model
Epoch 4/50
198/198 [==============================] - 3s 14ms/step - loss: 1.6132 - acc: 0.3838 - val_loss: 1.7530 - val_acc: 0.3134

Epoch 00004: val_loss did not improve from 1.73702
Epoch 5/50


Epoch 00032: val_loss did not improve from 0.30525
Epoch 33/50
198/198 [==============================] - 3s 13ms/step - loss: 0.0737 - acc: 0.9747 - val_loss: 0.4045 - val_acc: 0.8657

Epoch 00033: val_loss did not improve from 0.30525
Epoch 34/50
198/198 [==============================] - 3s 13ms/step - loss: 0.0929 - acc: 0.9747 - val_loss: 0.2977 - val_acc: 0.9104

Epoch 00034: val_loss improved from 0.30525 to 0.29770, saving model to C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/model/multi_img_classification.model
Epoch 35/50
198/198 [==============================] - 2s 12ms/step - loss: 0.0630 - acc: 0.9747 - val_loss: 0.2995 - val_acc: 0.8955

Epoch 00035: val_loss did not improve from 0.29770
Epoch 36/50
198/198 [==============================] - 3s 13ms/step - loss: 0.0411 - acc: 0.9899 - val_loss: 0.4285 - val_acc: 0.8657

Epoch 00036: val_loss did not improve from 0.29770
Epoch 37/50
198/198 [==============================] - 3s 13ms/step - loss: 0.0658 - acc: 0.9747 - v

In [63]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

67/67 [==============================] - 0s 4ms/step
정확도 : 0.9254


In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_oss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

In [ ]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

caltech_dir = "C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/multi_img_data/imgs_others_test"
image_w = 60
image_h = 60

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("LA")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('C:/Users/dahyun/Desktop/ARC_AutoDriving_Drone/model/multi_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    #print(i)
    #print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "a0"
    elif pre_ans == 1: pre_ans_str = "a1"
    elif pre_ans == 2: pre_ans_str = "b0"
    elif pre_ans == 3: pre_ans_str = "b1"
    elif pre_ans == 4: pre_ans_str = "c0"
    elif pre_ans == 5: pre_ans_str = "c1"
    if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[5] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    cnt += 1
    # print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
    # 즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
    # 이걸 한 것은 _4.py에.